In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import numpy as np
from copy import deepcopy

from nn_extrapolation import AcceleratedSGD

In [3]:
torch.cuda.is_available()

True

In [4]:
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.cuda()
            y = y.cuda()
            out = model(x)
            loss_sum += val_loss_fn(out, y)
            preds = out.argmax(1)
            ok += (y == preds).sum()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch(loss_log):
    model.train()
    for x, y in tqdm(train_loader):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss_log += list(loss.flatten().cpu().detach().numpy())
        loss.backward()
        optimizer.step()

In [5]:
train_ds = datasets.MNIST("../../../MNIST", download=True, train=True, transform=transforms.ToTensor())
test_ds = datasets.MNIST("../../../MNIST", download=True, train=False, transform=transforms.ToTensor())
valid_size = int(0.2 * len(train_ds))
train_ds, valid_ds = data.random_split(train_ds, [len(train_ds) - valid_size, valid_size])

train_loader = data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
valid_loader = data.DataLoader(valid_ds, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2)

In [6]:
def make_model():
    return nn.Sequential(
        nn.Conv2d(1, 32, 3),
        nn.ReLU(),
        nn.Conv2d(32, 32, 3),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Conv2d(32, 64, 3),
        nn.ReLU(),
        nn.Conv2d(64, 64, 3),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Flatten(),
        nn.Linear(4*4*64, 128),
        nn.ReLU(),
        nn.Linear(128, 10),
        nn.LogSoftmax(-1),
    )

In [7]:
model = make_model()
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [8]:
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, momentum=0.5, weight_decay=1e-5, mode="epoch")
loss_fn = nn.NLLLoss()

## Epoch

In [9]:
epochs = 20

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Training loss: 2.3038



Validation accuracy: 0.0952, validation loss: 2.3026
Epoch 2



Training loss: 2.3009



Validation accuracy: 0.0953, validation loss: 2.2996
Epoch 3


...


Validation accuracy: 0.9648, validation loss: 0.1144
Epoch 19



Training loss: 0.1017



Validation accuracy: 0.9678, validation loss: 0.1057
Epoch 20



Training loss: 0.0950



Validation accuracy: 0.9706, validation loss: 0.0986


In [10]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9732, device='cuda:0'), tensor(0.0873, device='cuda:0'))



Valid: (tensor(0.9706, device='cuda:0'), tensor(0.0986, device='cuda:0'))


In [11]:
%%time
optimizer.accelerate()

CPU times: user 622 ms, sys: 11.8 ms, total: 634 ms
Wall time: 243 ms


In [12]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()
None

In [13]:
print("Train:", validation(model_acc, train_loader))
print("Valid:", validation(model_acc, valid_loader))
model_acc.cpu()
None


Train: (tensor(0.9748, device='cuda:0'), tensor(0.0818, device='cuda:0'))



Valid: (tensor(0.9722, device='cuda:0'), tensor(0.0925, device='cuda:0'))


In [14]:
epochs = 10

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Training loss: 0.0899



Validation accuracy: 0.9711, validation loss: 0.0942
Epoch 2



Training loss: 0.0858



Validation accuracy: 0.9719, validation loss: 0.0904
Epoch 3


...


Validation accuracy: 0.9737, validation loss: 0.0784
Epoch 9



Training loss: 0.0646



Validation accuracy: 0.9772, validation loss: 0.0759
Epoch 10



Training loss: 0.0621



Validation accuracy: 0.9768, validation loss: 0.0736


In [15]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9815, device='cuda:0'), tensor(0.0590, device='cuda:0'))



Valid: (tensor(0.9768, device='cuda:0'), tensor(0.0736, device='cuda:0'))


In [16]:
optimizer.accelerate()
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()
None

In [17]:
print("Train:", validation(model_acc, train_loader))
print("Valid:", validation(model_acc, valid_loader))
model_acc.cpu()
None


Train: (tensor(0.9836, device='cuda:0'), tensor(0.0539, device='cuda:0'))



Valid: (tensor(0.9793, device='cuda:0'), tensor(0.0680, device='cuda:0'))


In [20]:
epochs = 5

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Training loss: 0.0590



Validation accuracy: 0.9794, validation loss: 0.0662
Epoch 2



Training loss: 0.0569



Validation accuracy: 0.9795, validation loss: 0.0684
Epoch 3


...


Validation accuracy: 0.9775, validation loss: 0.0722
Epoch 4



Training loss: 0.0532



Validation accuracy: 0.9790, validation loss: 0.0675
Epoch 5



Training loss: 0.0523



Validation accuracy: 0.9815, validation loss: 0.0620
